In [ ]:
!pip install transformers==4.28.0 datasets evaluate

In [2]:
import torch
import transformers
import pandas as pd
from tqdm.notebook import tqdm


In [4]:
file_path = 'All.conll'

dataset = pd.read_csv(file_path, delimiter='  ', skip_blank_lines=False)

<ipython-input-4-1dc56abf9b16>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset = pd.read_csv(file_path, delimiter='  ', skip_blank_lines=False)


In [52]:
dataset.head()

,word,tag
0,۵,O
1,طبقه,O
2,۳,B-Attributes of the property (A)
3,واحدی,I-Attributes of the property (A)
4,طبقه,B-Attributes of the property (A)


In [5]:
sentences = []
sentence = []
list_tags = []
list_tag = []
for index, row in dataset.iterrows():
    if pd.isna(row['word']):
        sentences.append(sentence)
        sentence = []
        list_tags.append(list_tag)
        list_tag = []
        pass
    else:
        sentence.append(row['word'])
        list_tag.append(row['tag'])

In [6]:
# Define the mapping from strings to numbers
mapping = {
    "O": 0,
    "B-Locality (L)": 1,
    "I-Locality (L)": 2,
    "B-Total Price (P)": 3,
    "I-Total Price (P)": 4,
    "B-Land Area (LA)": 5,
    "I-Land Area (LA)": 6,
    "B-Cost per land area (C)": 7,
    "I-Cost per land area (C)": 8,
    "B-Contact name (N)": 9,
    "I-Contact name (N)": 10,
    "B-Contact telephone (T)": 11,
    "I-Contact telephone (T)": 12,
    "B-Attributes of the property (A)": 13,
    "I-Attributes of the property (A)": 14
}

# Convert strings to numbers based on the mapping
list_tags = [[mapping[item] for item in list_tag] for list_tag in list_tags]

In [7]:
df = pd.DataFrame({'tokens': sentences, 'ner_tags': list_tags})
df.head()

,tokens,ner_tags
0,"[۵, طبقه, ۳, واحدی, طبقه, دوم, ۲, خوابه, ۲, سر...","[0, 0, 13, 14, 13, 14, 13, 14, 13, 14, 14, 13,..."
1,"[میدان, ولیعصر, چهار, پارکینگ, ۲۵۵, متر, زمین,...","[1, 2, 13, 14, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0..."
2,"[بهترین, خرید, و, سرمایه, گذاری, سال, فرعی, نف...","[0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 13, 14, 13, ..."
3,"[۴۳۰, مترمستغلات, سند, اداری, دسترسی, عالی, به...","[5, 6, 13, 14, 13, 14, 14, 14, 14, 14, 14, 13,..."
4,"[دید, و, نور, عالی, خیابان, اصلی, بسیار, خوش, ...","[13, 14, 14, 14, 1, 2, 0, 13, 14, 0, 0, 0, 1, ..."


In [8]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into train, test, and validation sets
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

# Reset the index of the DataFrames
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

# Print the number of samples in each set
print("Train set size:", len(train_df))
print("Test set size:", len(test_df))
print("Validation set size:", len(val_df))

Train set size: 446
Test set size: 56
Validation set size: 56


In [9]:
from datasets import Dataset, DatasetDict

import datasets
import pandas as pd

datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df),
    "validation": Dataset.from_pandas(val_df)
    })

In [10]:
datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 446
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 56
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 56
    })
})

In [ ]:
raw_datasets = datasets_train_test
raw_datasets["train"][0]["tokens"]

In [12]:
label_names = ["O", "B-Locality (L)", "I-Locality (L)",
    "B-Total Price (P)", "I-Total Price (P)", "B-Land Area (LA)", "I-Land Area (LA)",
    "B-Cost per land area (C)", "I-Cost per land area (C)", "B-Contact name (N)",
    "I-Contact name (N)", "B-Contact telephone (T)", "I-Contact telephone (T)",
    "B-Attributes of the property (A)", "I-Attributes of the property (A)"]

In [13]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

بزرگ‌ترین شبکه فروش و بروز‌ترین سامانه فایلینگ ملک در تبریز          با بیش از ۶۰ کارشناس فعال در سطح تبریز          شخصی ساز تک واحده دو انباری برای هر واحد مستر                             دار                              نما                              و                                مشاعات                           شیک                              آسانسور                          ۶ نفره پارکینگ                          اختصاصی پوشش کف                               سرامیک                           آماده تحویل فروشنده واقعی جهت کسب اطلاعات بیشتر تماس                    بگیرید                  کارشناس            منطقه              اسعدی              از دادن آدرس دقیق واحد به صورت تلفنی معذوریم آدرس           دفتر           چهارراه        منصور          پشت            برج            ابریشم         
O         O    O    O O         O      O       O   O  B-Locality (L) O  O   O  O  O       O    O  O   B-Locality (L) O    O   O  O     O  O      O    O  O    B-Attributes of the property (

In [14]:
from transformers import AutoTokenizer

model_checkpoint = "HooshvareLab/bert-fa-base-uncased-ner-arman"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [15]:
tokenizer.is_fast

True

In [16]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'بزرگترین',
 'شبکه',
 'فروش',
 'و',
 'بروزترین',
 'سامانه',
 'فایلی',
 '##نگ',
 'ملک',
 'در',
 'تبریز',
 'با',
 'بیش',
 'از',
 '۶۰',
 'کارشناس',
 'فعال',
 'در',
 'سطح',
 'تبریز',
 'شخصی',
 'ساز',
 'تک',
 'واحده',
 'دو',
 'انباری',
 'برای',
 'هر',
 'واحد',
 'مستر',
 'دار',
 'نما',
 'و',
 'مشاعات',
 'شیک',
 'اسانسور',
 '۶',
 'نفره',
 'پارکینگ',
 'اختصاصی',
 'پوشش',
 'کف',
 'سرامیک',
 'اماده',
 'تحویل',
 'فروشنده',
 'واقعی',
 'جهت',
 'کسب',
 'اطلاعات',
 'بیشتر',
 'تماس',
 'بگیرید',
 'کارشناس',
 'منطقه',
 'اسعدی',
 'از',
 'دادن',
 'ادرس',
 'دقیق',
 'واحد',
 'به',
 'صورت',
 'تلفنی',
 'معذور',
 '##یم',
 'ادرس',
 'دفتر',
 'چهارراه',
 'منصور',
 'پشت',
 'برج',
 'ابریشم',
 '[SEP]']

In [17]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 None]

In [18]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [19]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 13, 14, 14, 14, 13, 0, 0, 13, 0, 0, 13, 14, 0, 0, 0, 0, 0, 0, 0, 0, 11, 12, 9, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 13, 14, 14, 14, 13, 0, 0, 13, 0, 0, 13, 14, 0, 0, 0, 0, 0, 0, 0, 0, 11, 12, 9, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, -100]


In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [21]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/446 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [23]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,
            0,    0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    0,
            0,    0,    0,    0,    0,    0,   13,   14,   13,   14,   14,   14,
           13,    0,    0,   13,    0,    0,   13,   14,    0,    0,    0,    0,
            0,    0,    0,    0,   11,   12,    9,   10,   10,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    1,    2,    2,    2,    2,
            2,    2, -100, -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    1,    2,    2,    2,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,   13,   14,   14,   14,   14,
           14,    0,    0,    0,   13,   14,   14,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    9,
           10,    0,    0,    0,    0,    0, 

In [24]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 13, 14, 14, 14, 13, 0, 0, 13, 0, 0, 13, 14, 0, 0, 0, 0, 0, 0, 0, 0, 11, 12, 9, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 14, 14, 14, 14, 0, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 10, 0, 0, 0, 0, 0, 0, 0, 11, 12, 12, 9, 10, 10, 1, 2, 0, 0, 0, 0, -100]


In [ ]:
!pip install seqeval

In [27]:
import evaluate

metric = evaluate.load("seqeval")

In [28]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Locality (L)',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Locality (L)',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Attributes of the property (A)',
 'I-Attributes of the property (A)',
 'B-Attributes of the property (A)',
 'I-Attributes of the property (A)',
 'I-Attributes of the property (A)',
 'I-Attributes of the property (A)',
 'B-Attributes of the property (A)',
 'O',
 'O',
 'B-Attributes of the property (A)',
 'O',
 'O',
 'B-Attributes of the property (A)',
 'I-Attributes of the property (A)',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Contact telephone (T)',
 'I-Contact telephone (T)',
 'B-Contact name (N)',
 'I-Contact name (N)',
 'I-Contact name (N)',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Locality (L)',
 'I-Locality (L)',
 'I-Locality (L)',
 'I-Locality (L)',
 'I-Locality (L)',
 'I-Locality (L)',
 'I-Locality (L)']

In [29]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'Attributes of the property (A)': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 5},
 'Contact name (N)': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Contact telephone (T)': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 1},
 'Locality (L)': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [30]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [31]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [34]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased-ner-arman and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([13, 768]) in the checkpoint and torch.Size([15, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([13]) in the checkpoint and torch.Size([15]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
model.config.num_labels

15

In [41]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="ParsBERT_V2_ner_results",
    # evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=10,
    save_strategy="epoch",
    weight_decay=0.01,
    logging_steps=20,
    save_steps=50,
    logging_dir='ParsBERT_V2_ner_logs'
)

In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [43]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.115100
40,0.088900
60,0.071000
80,0.063800
100,0.044900
120,0.046200
140,0.032500
160,0.030000
180,0.027400
200,0.022100


TrainOutput(global_step=280, training_loss=0.04418019961033549, metrics={'train_runtime': 263.3124, 'train_samples_per_second': 16.938, 'train_steps_per_second': 1.063, 'total_flos': 369462936121020.0, 'train_loss': 0.04418019961033549, 'epoch': 10.0})

In [46]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 1.1688835620880127,
 'eval_precision': 0.4036697247706422,
 'eval_recall': 0.4661016949152542,
 'eval_f1': 0.4326450344149459,
 'eval_accuracy': 0.810604251253881,
 'eval_runtime': 0.8666,
 'eval_samples_per_second': 64.623,
 'eval_steps_per_second': 4.616,
 'epoch': 10.0}

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

In [48]:
import matplotlib.pyplot as plt

In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, confusion_matrix
def metrics_report(estimator, X_train, y_train, X_test, y_test, plot = True, ret = False):
  '''
  if you put ret to True it will return the following tupple:
    (accuracy_train, accuracy_test, precision_train, precision_test, recall_train, recall_test, f1_train, f1_test)
  '''
  pred_train = estimator.predict(X_train)
  pred_test = estimator.predict(X_test)

  accuracy_train = accuracy_score(y_train, pred_train)
  accuracy_test = accuracy_score(y_test, pred_test)

  precision_train = precision_score(y_train, pred_train, average = 'macro')
  precision_test = precision_score(y_test, pred_test, average = 'macro')

  recall_train = recall_score(y_train, pred_train, average = 'macro')
  recall_test = recall_score(y_test, pred_test, average = 'macro')

  f1_train = f1_score(y_train, pred_train, average = 'macro')
  f1_test = f1_score(y_test, pred_test, average = 'macro')

  if ret:
    return (accuracy_train, accuracy_test, precision_train, precision_test, recall_train, recall_test,
            f1_train, f1_test)

  else :
    print("accuracy on train = {:.3f}%".format(accuracy_train*100))
    print("accuracy on test = {:.3f}%".format(accuracy_test*100))

    print("\n")

    print("precision(macro) on train = {:.4f}".format(precision_train))
    print("precision(macro) on test = {:.4f}".format(precision_test))

    print("\n")

    print("recall(macro) on train = {:.4f}".format(recall_train))
    print("recall(macro) on test = {:.4f}".format(recall_test))

    print("\n")

    print("f1(macro) on train = {:.4f}".format(f1_train))
    print("f1(macro) on test = {:.4f}".format(f1_test))

    print("\n")

  if plot:
    disp = ConfusionMatrixDisplay(confusion_matrix(y_test, pred_test),
                      display_labels = np.unique(y_test))
    fig, ax = plt.subplots(figsize=(8, 8))
    disp.plot(ax=ax, cmap = 'rocket')
    plt.show()

In [59]:
trainer.predict(tokenized_datasets["test"])

PredictionOutput(predictions=array([[[ 1.2020697e+01, -1.4025915e-01, -6.9257349e-01, ...,
         -1.5389469e+00,  8.5514903e-01, -1.0977645e+00],
        [ 1.1871668e+01, -2.0649025e-01, -1.1577493e+00, ...,
         -1.1521171e+00,  2.4919942e-02, -7.5134313e-01],
        [ 1.1946114e+01, -5.2760065e-01, -4.3380740e-01, ...,
         -9.1286713e-01, -4.5989439e-01, -4.9187279e-01],
        ...,
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02]],

       [[ 1.2376743e+01, -3.9441329e-01, -5.1763678e-01, ...,
         -1.0360576e+00, -3.4563184e-02, -4.5010039e-01],
        [ 7.7464194e+00, -3.0868062e-01, -2.7854238e+00, ...,
         -1.5409726e+00,  2.9985464e+00, -2.2264423e+0